In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime

OIH = yf.download('OIH')['Adj Close']
RKH = yf.download('RKH')['Adj Close']
RTH = yf.download('RTH')['Adj Close']

df = pd.merge(OIH, RTH, on='Date', suffixes=('_OIH', '_RTH'))
df = pd.merge(df, RKH, on='Date')

df.rename(columns={"Adj Close" : "Adj Close_RKH"},inplace=True)

df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

Daily_ret = df.loc[:,('Adj Close_OIH','Adj Close_RTH','Adj Close_RKH')].pct_change()
Daily_ret.rename(columns={'Adj Close_OIH': 'OIH','Adj Close_RTH':'RTH','Adj Close_RKH':'RKH'},inplace=True)

Daily_ret


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,OIH,RTH,RKH
Date,,,
2001-05-17,NaN,NaN,NaN
2001-05-18,0.008198,-0.002551,-0.003441
2001-05-21,0.008975,0.022507,0.001295
2001-05-22,-0.015175,0.001501,0.018362
2001-05-23,-0.037514,-0.018981,-0.001100
...,...,...,...
2014-12-10,-0.035155,-0.011082,-0.018659
2014-12-11,-0.003700,0.014700,0.007534
2014-12-12,-0.023714,-0.003156,-0.026705


In [14]:
ExcessRet = Daily_ret-0.04/252
M =252*ExcessRet.mean()
M

OIH    0.044365
RTH    0.050926
RKH   -0.065439
dtype: float64

In [15]:
C = 252*ExcessRet.cov()
C

,OIH,RTH,RKH
OIH,0.146408,0.034629,0.080959
RTH,0.034629,0.045887,0.041433
RKH,0.080959,0.041433,0.194883


In [18]:
#Optimazation dot product ( inverse Covariance Matrix * Mean Excess returns)
from numpy.linalg import inv
F = np.dot(inv(C),M) 
F

array([ 0.39287686,  1.56416652, -0.83154648])

In [20]:
#long term growth g = risk free + optimazation vector transposed * ( Cov matrix * Optimazatino vector) / 2
g = 0.04 + np.dot(F.T, np.dot(C, F))/2
g

0.11575141552636889

In [22]:
S = np.sqrt(np.dot(F.T, np.dot(C,F)))
S

0.3892336458385089